# Setup

In [2]:
remote = False

from google.colab import files
files.upload()  # Upload your kaggle.json here.

from google.colab import drive
drive.mount('/content/gdrive')

# In the scenario where we work away from laptop.
import shutil
gdrive_kggle_json_path = r"/content/gdrive/MyDrive/Kaggle Competitions/kaggle.json"
if remote:
  shutil.copy(gdrive_kggle_json_path, ".")

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# !mkdir kaggle/
# %cd kaggle/
# !mkdir input/
# !mkdir output/
# %cd input/

!mkdir rsna-miccai-processed
%cd rsna-miccai-processed/
!kaggle datasets download -d sreevishnudamodaran/rsna-processed-voxels-64x256x256-clahe
!unzip -q rsna-processed-voxels-64x256x256-clahe.zip
!rm rsna-processed-voxels-64x256x256-clahe.zip
%cd ../

# %cd output/

Saving kaggle.json to kaggle.json
Mounted at /content/gdrive
/content/rsna-miccai-processed
100% 3.49G/3.50G [00:28<00:00, 102MB/s]
100% 3.50G/3.50G [00:28<00:00, 134MB/s]
/content


In [3]:
!pip install albumentations --upgrade -q
!pip install pydicom -q
!pip install timm -q
!pip install torch --upgrade -q
!pip install torchio -q
!pip install tensorflow_addons -q
!pip install keras_applications -q
!pip install efficientnet-3D -q
!pip install git+https://github.com/shijianjian/EfficientNet-PyTorch-3D -q
!pip install classification-models-3D -q
!pip install volumentations-3D -q
!pip install --upgrade https://github.com/VincentStimper/mclahe/archive/numpy.zip -q

     |████████████████████████████████| 98 kB 3.9 MB/s 
     |████████████████████████████████| 37.1 MB 87 kB/s 
     |████████████████████████████████| 2.0 MB 5.1 MB/s 
     |████████████████████████████████| 376 kB 5.0 MB/s 
     |████████████████████████████████| 831.4 MB 5.2 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.9.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.9.1 which is incompatible.
     |████████████████████████████████| 156 kB 5.2 MB/s 
     |████████████████████████████████| 48.4 MB 8.6 kB/s 
     |████████████████████████████████| 1.1 MB 5.3 MB/s 
     |████████████████████████████████| 50 kB 2.8 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 
     | 7.5 MB 1.9 MB/s


In [4]:
# General imports.
import gc
import re
import sys
import PIL
import yaml
import math
import random
import pydicom
import platform
import shutil, os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import albumentations as A
import volumentations as V
import mclahe as mc
import sklearn
import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchio as tio
import tensorflow as tf

# Specific imports. 
from math import ceil
from glob import glob
from PIL import Image
from IPython.display import Image, clear_output
from pydicom.pixel_data_handlers.util import apply_voi_lut

from tqdm.notebook import tqdm
from matplotlib import animation, rc
rc('animation', html='jshtml')
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GroupKFold
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow.experimental.numpy as tnp
from efficientnet_3D import keras as efn  # TF.keras.
from efficientnet_pytorch_3d import EfficientNet3D  # Pytorch.
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torchsummary import summary
from torchvision import transforms

import warnings
warnings.simplefilter('ignore')

In [5]:
!pip install wandb -qqq
import wandb
# from wandb.keras import WandbCallback
wandb.login()

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 97 kB 5.8 MB/s 
     |████████████████████████████████| 139 kB 45.7 MB/s 
     |████████████████████████████████| 180 kB 48.4 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Utility Functions

In [6]:
def seed_everything(seed=123):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

In [7]:
def create_animation(ims):
    fig = plt.figure(figsize=(6, 6))
    plt.axis('off')
    im = plt.imshow(ims[0], cmap="gray")

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)

# Hyperparameters

In [8]:
class Config:
  model_name = "efficientnet-b0"
  seq_length = 64
  batch_size = 10
  image_size = 256
  train_size = 0.85
  val_size = 1.0 - train_size
  n_splits = 5
  n_epochs = 10
  init_lr = 0.0001
  in_channels = 1

  FLIP = 0.5 # @params: probability
  CONTRAST = (0.7, 1.3, 0.5) # @params: (minval, maxval, probability)
  BRIGHTNESS = (0.2, 0.4) # @params: (delta, probability)
  GAMMA = (0.8, 1.2, 0.5) # @params: (minval, maxval, probability)
  ROTATE = (20, 0.5) # @params: (maxangle, probability)
  RANDOM_CROP = (180, 180, 0.4) # @params: (min_width, min_height, probability)
  CUTOUT = ((15, 15), 15, 0.4) # @params: ((mask_dim0, mask_dim1), max_num_holes, probability)
  BLUR = ([5, 5], 10, 0.4) # @params: ((filter_dim0, filter_dim1), sigma, probability)

  # Permanent parameters.
  num_workers = 0
  folds = 5
  classes = ["MGMT_value"]
  n_classes = len(classes)
  project_name_1fold = f"RSNA-MICCAI-efnb0-3d_{image_size}_1fold"
  project_name_5fold = f"RSNA-MICCAI-efnb0-3d_{image_size}_5fold"
  images_dir = r"/content/rsna-miccai-processed/voxels"
  ckpt_dir = "."
  seed = 123
  seed_everything(seed)

# Dataset

## Augmentations 3D

In [9]:
def random_rotate3D(voxel, limit=90, p=0.5):
  if tf.random.uniform(()) < p:
      angle = tf.random.uniform((), minval=-limit, maxval=limit,
                                dtype=tf.int32)
      voxel = tfa.image.rotate(voxel, tf.cast(angle,
                                              tf.float32)*(math.pi/180),
                                interpolation='nearest',
                                fill_mode='constant',
                                fill_value=0.0)
  return voxel

def random_resized_crop3D(voxel, min_width, min_height, p=0.5):
  if tf.random.uniform(()) < p:
      voxel_shape = voxel.shape
      assert voxel_shape[1] >= min_height
      assert voxel_shape[2] >= min_width
          
      width = tf.random.uniform((), minval=min_width,
                                maxval=voxel_shape[2],
                                dtype=tf.int32)
      height = tf.random.uniform((), minval=min_height,
                                maxval=voxel_shape[1],
                                dtype=tf.int32)
      x = tf.random.uniform((), minval=0,
                            maxval=voxel_shape[2] - width,
                            dtype=tf.int32)
      y = tf.random.uniform((), minval=0,
                            maxval=voxel_shape[1] - height,
                            dtype=tf.int32)
      voxel = voxel[:, y:y+height, x:x+width, :]
      voxel = tf.image.resize(voxel,
                              voxel_shape[1:3],
                              method='lanczos5')
  return voxel

def random_cutout3D(voxel, mask_shape=(10, 10), num_holes=20, p=0.5):
    if tf.random.uniform(()) < p:
        voxel_shape = voxel.shape
        assert voxel_shape[1] >= mask_shape[0]
        assert voxel_shape[2] >= mask_shape[1], f"{voxel_shape} voxel shape and {mask_shape} mask shape"

        holes = tf.random.uniform((), minval=1, maxval=num_holes,
                                  dtype=tf.int32)
        mask_size = tf.constant([mask_shape[0], mask_shape[1]])
        mask = tf.Variable((lambda : tf.ones(voxel_shape)),
                          trainable=False)
          
        for i in tf.range(holes):
            x = tf.random.uniform((), minval=0,
                                  maxval=voxel_shape[2],
                                  dtype=tf.int32)
            y = tf.random.uniform((), minval=0,
                                  maxval=voxel_shape[1],
                                  dtype=tf.int32)
            mask_endx = tf.add(x, mask_size[1])
            mask_endy = tf.add(y, mask_size[0])
            mask[:, x:mask_endx,
                y:mask_endy, :].assign(tf.zeros_like(mask[:, x:mask_endx,
                                                        y:mask_endy, :]))
        voxel = tf.Tensor(np.multiply(voxel.numpy(), mask.numpy()))
        mask.assign(tf.ones(voxel_shape))
    return voxel

def _get_gaussian_kernel(sigma, filter_shape):
    x = tf.range(-filter_shape // 2 + 1, filter_shape // 2 + 1)
    x = tf.cast(x ** 2, sigma.dtype)
    x = tf.nn.softmax(-x / (2.0 * (sigma ** 2)))
    return x

def random_gaussian_blur3D(voxel, filter_shape=[5, 5], max_sigma=3, p=0.5):
    if tf.random.uniform(()) < p:
        sigma = tf.random.uniform((), minval=3, maxval=max_sigma,
                          dtype=tf.int32)
        filter_shape = tf.constant(filter_shape)
        channels = voxel.shape[-1]
        sigma = tf.cast(sigma, voxel.dtype)
        gaussian_kernel_x = _get_gaussian_kernel(sigma,
                                                filter_shape[1])
        gaussian_kernel_x = gaussian_kernel_x[tf.newaxis, :]
        gaussian_kernel_y = _get_gaussian_kernel(sigma,
                                                filter_shape[0])        
        gaussian_kernel_y = gaussian_kernel_y[:, tf.newaxis]
        gaussian_kernel_2d = tf.matmul(gaussian_kernel_y,
                                      gaussian_kernel_x)
        gaussian_kernel_2d = gaussian_kernel_2d[:, :, tf.newaxis,
                                                tf.newaxis]
        gaussian_kernel_2d = tf.tile(gaussian_kernel_2d,
                                    tf.constant([1, 1, channels, 1]))
        voxel = tf.nn.depthwise_conv2d(input=voxel,
                                      filter=gaussian_kernel_2d,
                                      strides=(1, 1, 1, 1),
                                      padding="SAME",
                                      )
        voxel = tf.cast(voxel, voxel.dtype)
    return voxel

## Building the Dataset

In [10]:
train_val_labels = pd.read_csv(r"/content/gdrive/MyDrive/Kaggle Competitions/RSNA-brain-tumor/train_labels.csv")
train_val_labels.head()

,BraTS21ID,MGMT_value
0,0,1
1,2,1
2,3,0
3,5,1
4,6,1


In [11]:
train_val_labels = train_val_labels[train_val_labels.BraTS21ID != 109]
train_val_labels = train_val_labels[train_val_labels.BraTS21ID != 123]
train_val_labels = train_val_labels[train_val_labels.BraTS21ID != 709]
train_val_labels = train_val_labels.reset_index(drop=True)

In [12]:
# 1-Fold.
cfg = Config()

sss = StratifiedShuffleSplit(n_splits=1, train_size=cfg.train_size)

for train_idx, val_idx in sss.split(train_val_labels, train_val_labels.MGMT_value):
  train_labels_1fold = train_val_labels.loc[train_idx].reset_index(drop=True)
  val_labels_1fold = train_val_labels.loc[val_idx].reset_index(drop=True)

In [13]:
train_labels_1fold.shape, val_labels_1fold.shape

((494, 2), (88, 2))

In [14]:
display(train_labels_1fold.MGMT_value.value_counts()),
display(val_labels_1fold.MGMT_value.value_counts())

1    260
0    234
Name: MGMT_value, dtype: int64

1    46
0    42
Name: MGMT_value, dtype: int64

In [15]:
# 5-Fold.
skf = StratifiedKFold(n_splits=cfg.n_splits)

train_val_labels_5fold = train_val_labels.copy()
for fold, train_val_idx in enumerate(skf.split(train_val_labels_5fold, train_val_labels_5fold.MGMT_value)):
  train_idx = train_val_idx[0]
  val_idx = train_val_idx[1]
  train_val_labels_5fold.loc[val_idx, "skf_mgmt_fold"] = fold

In [16]:
train_val_labels_5fold.skf_mgmt_fold.value_counts()

1.0    117
0.0    117
4.0    116
3.0    116
2.0    116
Name: skf_mgmt_fold, dtype: int64

In [17]:
train_labels_1fold

,BraTS21ID,MGMT_value
0,549,1
1,559,1
2,339,0
3,436,1
4,737,1
...,...,...
489,138,1
490,390,0
491,300,0
492,325,0


In [18]:
# 1-Fold.
class ProcessedTrain1FoldDataset(Dataset, ):
    def __init__(self, df, images_dir, image_size, modality, mode, classes):
        super(ProcessedTrain1FoldDataset, self).__init__()
        self.df = df.reset_index(drop=True)
        self.images_dir = images_dir
        self.image_size = image_size
        self.modality = modality
        self.modality_dir = os.path.join(self.images_dir, self.modality)
        assert mode in ['train', 'valid']
        self.mode = mode
        self.classes = classes

        if self.mode == 'train':
            # self.df = self.df.sample(frac=1).reset_index(drop=True)
            self.transform = tio.Compose([
              tio.RandomElasticDeformation(p=0.5),
              tio.RandomFlip(p=0.5),
              tio.RandomGamma(p=0.5),
              tio.RandomNoise(p=0.5),
              tio.RandomAffine(p=0.5),
              tio.RandomAnisotropy(p=0.5),
              tio.RandomBlur(p=0.5),
              tio.RandomMotion(p=0.5),
              tio.RandomElasticDeformation(p=0.5),
              tio.RandomSpike(p=0.5),
              tio.RandomGhosting(p=0.5),
              tio.ZNormalization()
            ])
        else:
            self.transform = tio.Compose([
              tio.ZNormalization()
            ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
          patientid = self.df.loc[index, "BraTS21ID"]
          label = torch.FloatTensor(self.df.loc[index, self.classes])

          # Maybe 3 models: 1 for FLAIR, 1 for T1wCE, 1 for T2w (exclude T1w).
          img_npy_path = os.path.join(self.modality_dir, str(patientid).zfill(5) + ".npy")
          img = np.load(img_npy_path).clip(0, 255)
          img = np.expand_dims(img, axis=-1)
          # img = np.stack([img, img, img], axis=-1)

          img = np.transpose(img, [3, 1, 2, 0])

          img = self.transform(data=img)

          img = np.transpose(img, [0, 3, 1, 2])

          return {"images": torch.FloatTensor(img), "labels": label}

In [19]:
# ani = create_animation(np.transpose(np.squeeze(exp_ds[0]["images"]), [2, 0, 1]))
# ani

In [20]:
# writervideo = animation.FFMpegWriter(fps=60)
# ani.save("a_gif.mp4", writer=writervideo)

# Training 1-Fold

In [21]:
import torch
if torch.cuda.is_available():
  print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
else:
  print("\n[INFO] GPU not found. Using CPU: {}\n".format(platform.processor()))

[INFO] Using GPU: Tesla P100-PCIE-16GB



In [ ]:
cfg = Config()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

training_modalities = ["FLAIR", "T1wCE", "T2w"]

models = {modality: EfficientNet3D.from_name(cfg.model_name, override_params={'num_classes': cfg.n_classes}, in_channels=cfg.in_channels) for modality in training_modalities}

for modality in training_modalities:
  CHECKPOINT = '{}/{}_{}_1fold_{}.pth'.format(cfg.ckpt_dir, cfg.model_name, cfg.image_size, modality)

  run = wandb.init(project=cfg.project_name_1fold, name=f"{modality}")

  train_ds = ProcessedTrain1FoldDataset(df=train_labels_1fold, 
                                    images_dir=cfg.images_dir, 
                                    image_size=cfg.image_size, 
                                    modality=modality,
                                    mode="train", 
                                    classes=cfg.classes)
  val_ds = ProcessedTrain1FoldDataset(df=val_labels_1fold, 
                                    images_dir=cfg.images_dir, 
                                    image_size=cfg.image_size, 
                                    modality=modality,
                                    mode="valid", 
                                    classes=cfg.classes)
  
  train_loader = DataLoader(train_ds, 
             batch_size=cfg.batch_size, 
             sampler=RandomSampler(train_ds), 
             num_workers=cfg.num_workers)

  val_loader = DataLoader(val_ds, 
             batch_size=cfg.batch_size, 
             sampler=SequentialSampler(val_ds), 
             num_workers=cfg.num_workers)

  model = models[modality]
  if modality == "FLAIR":
    model.load_state_dict(torch.load(r"/content/0ce0f5b3f7abc9cfcb439d540d35d37e"))
  criterion = nn.BCEWithLogitsLoss()
  scaler = torch.cuda.amp.GradScaler()
  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.init_lr)
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, cfg.n_epochs-1)

  val_loss_min = np.Inf
  early_stopping_patience = 5
  cnt = 0

  for idx, epoch in enumerate(range(1, cfg.n_epochs + 1)):
      scheduler.step()
      model.train().to(device)
      train_loss = []

      print('Epoch: {:02d}/{:02d}'.format(epoch, cfg.n_epochs))
      print("TRAIN")

      loop = tqdm(train_loader)
      for batch in loop:
        images = batch["images"]
        labels = batch["labels"]
        images = images.to(device).float()
        labels = labels.to(device).float()

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
          output = model(images)
          loss = criterion(output, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss.append(loss.item())
        loop.set_description('current_loss: {:.5f} | LR: {:.5f}'.format(loss.item(), optimizer.param_groups[0]['lr']))
        loop.set_postfix(loss=np.mean(train_loss))
      train_loss = np.mean(train_loss)

      model.eval()

      auc_scores = []
      val_loss = []

      print("VAL")
      loop = tqdm(val_loader)
      for batches in loop:
        images = batch["images"]
        labels = batch["labels"]
        images = images.to(device).float()
        labels = labels.to(device).float()

        with torch.cuda.amp.autocast(), torch.no_grad():
          outputs = model(images)
          loss = criterion(outputs.float(), labels)
          
        auc_score = roc_auc_score(labels.data.cpu().numpy(), 
                                  F.sigmoid(outputs).data.cpu().numpy())
        auc_scores.append(auc_score)

        val_loss.append(loss.item())
        loop.set_description('current_loss: {:.5f} | current_auc: {:.5f}'.format(loss.item(), auc_score))
        loop.set_postfix(loss=np.mean(val_loss), auc=np.mean(auc_scores))
      val_loss = np.mean(val_loss)
      final_auc_score = np.mean(auc_scores)

      wandb.log({"epoch": epoch, 
                "loss": train_loss, 
                "val_loss": val_loss,
                "val_auc": final_auc_score,
                })
      
      if val_loss < val_loss_min:
        cnt = 0
        print('Valid loss improved from {:.5f} to {:.5f} saving model to {}'.format(val_loss_min, val_loss, CHECKPOINT))
        val_loss_min = val_loss
        torch.save(model.state_dict(), CHECKPOINT)
        artifact = wandb.Artifact(cfg.model_name, type='model')
        artifact.add_file(CHECKPOINT, name=f"{modality}_epoch_{epoch}.pt")
        run.log_artifact(artifact)
      else:
        cnt += 1
      print("")
      
      if cnt >= early_stopping_patience:
        break 

  del model
  del optimizer
  gc.collect()
  torch.cuda.empty_cache()

  run.finish()

wandb: Currently logged in as: vincenttu (use `wandb login --relogin` to force relogin)


Epoch: 01/10
TRAIN


  0%|          | 0/50 [00:00<?, ?it/s]